# A testing system of ´*.pt´-files

Based only on the `model.pt` file we can calculate stats from our standardized set [Roboflow project](https://universe.roboflow.com/ntnu-3oxpl/testset-qs9tl)
This is just a hardcoded example set: The solution array is made by hand. The first column is the main building color and the second is all the colors present. By comparing the solution to the output of the test file we can make some stats.
In order to use this test, just switch the `v12_larger.pt` file with your own model. Give the name in the box below and run the rest of the cells. The notebook will print the stats for the model at the end.

In [ ]:
modelname = "v12_larger.pt"
pathToModel = f"data/models/{modelname}"

This is the solution of the test set:
It checks for the following things:
1. What color the house in focus  is.
2. How many houses there are and which types

Other things: 
     It contains 20 images. Not used for anything in training our model. It is not part of the roboflow-dataset. It was only created for comparing models easily.

|Labels|white|red|orange|black|yellow|darkgrey|grey|brown|blue|green|pink|
|------|-----|---|------|-----|------|--------|----|-----|----|-----|----|
|Amount|21|9|8|6|6|3|3|2|1|1|1|

In [ ]:
solution = []

# A hard-coded solution for the standard test set. First the main house, then all the present houses: 
solution.append(["Main house color", "list of colors in picture"])

solution.append(["white", {'black':2, 'darkgrey':1, 'grey':1, 'white':1, 'yellow':1}])
solution.append(["orange", {'orange':1, 'red':1, 'white':1, 'yellow':1}])
solution.append(["red", {'orange':1, 'red':1, 'white':1}])
solution.append(["black", {'black':1, 'orange':1, 'yellow':1, 'white':1}])
solution.append(["empty", {}])
solution.append(["white", {'white':1}])
solution.append(["yellow", {'yellow':2}])
solution.append(["red", {'red':4, 'white':1}])
solution.append(["orange", {'orange':1}])
solution.append(["brown", {'brown':1, 'grey':1, 'red':1}])
solution.append(["white", {'white':3}])
solution.append(["darkgrey", {'darkgrey':1, 'white':1}])
solution.append(["grey", {'darkgrey':1, 'grey':1, 'red':1, 'white':1}])
solution.append(["black", {'orange':1, 'black':2, 'white':2, 'red':1}])
solution.append(["black", {'black':1}])
solution.append(["yellow", {'orange':1, 'green':1, 'white':2, 'yellow':1}])
solution.append(["white", {'white':4}])
solution.append(["brown", {'brown':1}])
solution.append(["blue", {'blue':1}])
solution.append(["white", {'orange':2, 'pink':1, 'white':1}])



Stats returned from the test:
1. Correct & false amount of main house predictions
2. Correct & false amount of total houses predictions
3. How many houses were missed

Will not print out the images, but will print out the total stats. If you want to see the images, just remove the `#` in the last cell.

In [ ]:
# import modules
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image
import os

from tabulate import tabulate

In [ ]:
# Change path, at the top of the notebook
model = YOLO(pathToModel)

### We need some functions in order to retrieve the results:

In [ ]:
def gradeResults(result):
    predictions = []
    
    imgWidth = result[0].orig_shape[0]
    imgHeight = result[0].orig_shape[1]
    
    for box in result[0].boxes:
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        
        x, y, w, h = box.xywh[0]
        x_offset = imgWidth/2 - (x)
        y_offset = imgHeight/2 - (y)
        offset =  x_offset**2 + y_offset**2
        predictions.append((predictedClass, confidence, offset))
    # Outputs list of guesses, confidence, and offset from center.
    return predictions

def chooseTheMiddleOne(predictions):
    if len(predictions) == 0:
        return "empty"
    # Sorts by offset from center
    predictions.sort(key=lambda x: x[2])
    
    minAcceptableOffset = 300
    if predictions[0][2] > minAcceptableOffset**2:
        return "bad"
    
    MiddleBoxClass = predictions[0][0]
    return MiddleBoxClass

### Predicting all the images in the folder

In [ ]:
MyPredictions = [["Main building", "colors in picture"]]
for index in range(1, 21):
    img = "data/buildings/standardized_test/" + str(index) + ".png"
    result = model(img, conf=0.35)
    predictions = gradeResults(result)
    colors = dict()
    # Count the colors of the boxes
    for prediction in predictions:
        if prediction[0] in colors:
            colors[prediction[0]] += 1
        else:
            colors[prediction[0]] = 1    
    middleHousePrediction = chooseTheMiddleOne(predictions)
    
    MyPredictions.append([middleHousePrediction, colors])


In [ ]:
# Printing a comparison table
print(tabulate(zip(MyPredictions, solution), headers=["The prediction", "The solution"]))

In [ ]:
class Stats:
    def __init__(self, pred_, sol_):
        self.pred = pred_
        self.sol = sol_
        
        self.numberOfBuildings = 61
        self.numberOfImages = 20
                
    def ComparePredAndSol(imgPred: dict, imgSol: dict):
        correct = 0
        missed = 0
        tooMany = 0
        
        totalBuildings = 0
        totalBuildingsPredicted = 0
        
        # Counting the total number of buildings predicted
        for key in imgPred:
            totalBuildingsPredicted += imgPred[key]
        
        # Counting the total number of buildings in the solution and comparing with prediction
        for key in imgSol:
            totalBuildings += imgSol[key]
            
            if key in imgPred:
                if imgPred[key] == imgSol[key]:
                    correct += imgSol[key]
                else:
                    correct += min(imgPred[key], imgSol[key])   
        
        missed = max(totalBuildings - totalBuildingsPredicted, 0)  
        tooMany = max(0, totalBuildingsPredicted - totalBuildings)  
        return correct, missed, tooMany   
                
    def calculateAccuracy(self):
        # Main building calculation
        self.CorrectMainBuildings = 0
        for index in range(1, 21):
            if self.pred[index][0] == self.sol[index][0]:
                self.CorrectMainBuildings += 1
                
        self.CorrectBuildings = 0
        self.MissedBuildings = 0
        self.TooManyBuildings = 0
        for index in range(1, self.numberOfImages + 1):
            # Adding stats one image at a time
            correct, missed, tooMany = Stats.ComparePredAndSol(self.pred[index][1], self.sol[index][1])
            self.CorrectBuildings += correct
            self.MissedBuildings += missed
            self.TooManyBuildings += tooMany
        
    def printStats(self):
        statTable = []
        
        mainBuildAccStat = str(100 * round(self.CorrectMainBuildings/self.numberOfImages, 3))
        accStat = str(100 * round(self.CorrectBuildings/self.numberOfBuildings, 3))
        missStat = str(100 * round(self.MissedBuildings/self.numberOfBuildings, 3))
        tooManyStat = str(100 * round(self.TooManyBuildings/self.numberOfImages, 3))
        
        statTable.append(["Main building accuracy", str(self.CorrectMainBuildings), f"{str(self.numberOfImages)} Images", mainBuildAccStat])
        statTable.append(["Accuracy", str(self.CorrectBuildings), f"{str(self.numberOfBuildings)} Buildings", accStat])
        statTable.append(["Missed", str(self.MissedBuildings), f"{str(self.numberOfBuildings)} Buildings", missStat])
        statTable.append(["Too many", str(self.TooManyBuildings), f"{str(self.numberOfImages)} Images", tooManyStat])                 
        
        header = ["Statistic", "Result", "Out of", "Percentage"]
        
        print(tabulate(statTable, headers=header))
        
        # Alternative way of printing without the tabulate method
        # print(f"Main building accuracy: {str(self.CorrectMainBuildings)}/{str(self.numberOfImages)}, {str(round(self.CorrectMainBuildings/self.numberOfImages, 3))}%")
        # print(f"Accuracy: {str(self.CorrectBuildings)}/{str(self.numberOfBuildings)}, {str(round(self.CorrectBuildings/self.numberOfBuildings, 3))}%")
        # print(f"Missed: {str(self.MissedBuildings)}/{str(self.numberOfBuildings)}, {str(round(self.MissedBuildings/self.numberOfBuildings, 3))}%")
        # print(f"Too many: {str(self.TooManyBuildings)}/{str(self.numberOfImages)}, {str(round(self.TooManyBuildings/self.numberOfImages, 3))}%")
        

In [ ]:
stats = Stats(MyPredictions, solution)
stats.calculateAccuracy()
stats.printStats()

To inspect the results, uncomment the following line
That is to inspect the boxes
The result should be shown inside a new folder named `outputs\standardized_test`	

In [ ]:
# confidence_threshold = 0.25
# model("data/buildings/standardized_test/", conf=confidence_threshold, save=True, project="./outputs", name="standardized_test")